<a href="https://colab.research.google.com/github/NahidurRahaman/Machine_Learning/blob/main/project_5_ocr_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning
import os
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from glob import glob
from torchmetrics import Accuracy
from torchvision import models

In [ ]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
ALL_CHAR_SET = NUMBER + ALPHABET
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 4

In [ ]:
ALL_CHAR_SET_LEN

36

# Data

In [ ]:
class OCRDataset(Dataset):
    def __init__(self, image_files, transform=None):
        self.image_files = image_files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def encode(self, char):
        onehot = [0] * ALL_CHAR_SET_LEN
        idx = ALL_CHAR_SET.index(char)
        onehot[idx] += 1
        return onehot

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        img = Image.open(image_file)
        img = img.convert('L')
        label = image_file.split(os.sep)[-1][:-4]
        label_oh = []
        for i in label[:MAX_CAPTCHA]:  # Ensure label length <= 5
            label_oh += self.encode(i)
        # Pad with zeros if label is shorter than MAX_CAPTCHA
        label_oh += [0] * (ALL_CHAR_SET_LEN * (MAX_CAPTCHA - len(label)))
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class OCRDataModule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size

        self.transform = transforms.Compose([
            transforms.Resize([64, 128]),
            transforms.ToTensor(),
        ])

    def setup(self, stage=None):
        train_image_files = glob(r'/content/drive/MyDrive/BongoDev/Project-1/OCR/Train/*')
        test_image_files = glob(r'/content/drive/MyDrive/BongoDev/Project-1/OCR/Test/*')

        self.train_dataset = OCRDataset(
            train_image_files,
            transform=self.transform
        )

        self.val_dataset = OCRDataset(
            test_image_files,
            transform=self.transform
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.val_dataset, # Using val_dataset for testing as per the current setup
            batch_size=self.batch_size,
            shuffle=False,
        )

In [ ]:
data_module = OCRDataModule(
    batch_size=32
)

data_module.setup()

In [ ]:
for batch in data_module.train_dataloader():
    images, encoded_labels, labels = batch
    print(images.shape)  # Should be (32, 1, 64, 128)
    print(encoded_labels.shape)  # Should be (32, 180)
    print(labels)  # List of 32 strings
    break

torch.Size([32, 1, 64, 128])
torch.Size([32, 144])
('6g46', 'awf5', 'g8wj', 'l63x', '0tx7', '3wge', '3l9j', '2m6c', '7cbp', '3jxh(1)', 'h9nw', '5apz', '0ws3', '9xxp', '8s2c', '30yk', '6th4', 'gz7y', '18si', '6p7b', 'h94e', 'hlx9', 'u82e', '6a99', 'j2j6', '9tft', 'w37f', '3l9j(1)', 'ld1l', 'gym0', '0wlz', '01se')


# Model

In [ ]:
class OCRModel(pl.LightningModule):
    def __init__(self, num_chars=ALL_CHAR_SET_LEN, max_length=MAX_CAPTCHA):
        super(OCRModel, self).__init__()
        self.save_hyperparameters()
        self.char_set = ALL_CHAR_SET

        # Load pretrained ResNet18
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

        # Modify the first conv layer for 1-channel input
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Modify the final fully connected layer
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_chars * max_length)
        )

        self.train_accuracy = Accuracy(task='multiclass', num_classes=num_chars)
        self.val_accuracy = Accuracy(task='multiclass', num_classes=num_chars)
        self.test_accuracy = Accuracy(task='multiclass', num_classes=num_chars)

        # Transform for inference
        self.inference_transform = transforms.Compose([
            transforms.Resize([64, 128]),
            transforms.ToTensor(),
        ])

    def forward(self, x):
        out = self.resnet(x)
        return out.view(-1, self.hparams.max_length, self.hparams.num_chars)

    def training_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)

        # Reshape for cross_entropy
        batch_size = x.size(0)
        y_hat = y_hat.view(batch_size * self.hparams.max_length, self.hparams.num_chars)
        y = y.view(batch_size, self.hparams.max_length, self.hparams.num_chars)
        y = y.argmax(dim=-1).view(-1)  # Convert one-hot to indices

        loss = F.cross_entropy(y_hat, y)

        self.train_accuracy(y_hat.softmax(dim=-1), y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', self.train_accuracy, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)

        # Reshape for cross_entropy
        batch_size = x.size(0)
        y_hat = y_hat.view(batch_size * self.hparams.max_length, self.hparams.num_chars)
        y = y.view(batch_size, self.hparams.max_length, self.hparams.num_chars)
        y = y.argmax(dim=-1).view(-1)  # Convert one-hot to indices

        loss = F.cross_entropy(y_hat, y)

        self.val_accuracy(y_hat.softmax(dim=-1), y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.val_accuracy, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)

        # Reshape for cross_entropy
        batch_size = x.size(0)
        y_hat = y_hat.view(batch_size * self.hparams.max_length, self.hparams.num_chars)
        y = y.view(batch_size, self.hparams.max_length, self.hparams.num_chars)
        y = y.argmax(dim=-1).view(-1)  # Convert one-hot to indices

        loss = F.cross_entropy(y_hat, y)

        self.test_accuracy(y_hat.softmax(dim=-1), y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', self.test_accuracy, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler
        }

    def predict_image(self, image_path):
        """Perform inference on a single image file and return the predicted text."""
        # Load and preprocess image
        img = Image.open(image_path).convert('L')
        img = self.inference_transform(img)
        img = img.unsqueeze(0)  # Add batch dimension: (1, 1, 64, 128)

        # Move to device
        img = img.to(self.device)

        # Set model to evaluation mode
        self.eval()
        with torch.no_grad():
            # Forward pass
            output = self(img)  # Shape: (1, 5, 36)
            output = output.softmax(dim=-1)  # Convert to probabilities
            pred_indices = output.argmax(dim=-1)  # Shape: (1, 5)

        # Decode indices to characters
        pred_text = ''.join(self.char_set[idx] for idx in pred_indices[0])
        return pred_text

In [ ]:
model = OCRModel()

In [ ]:
trainer = pl.Trainer(
        max_epochs=50,
)
trainer.fit(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ resnet         │ ResNet             │ 11.5 M │ train │     0 │
│ 1 │ train_accuracy │ MulticlassAccuracy │      0 │ train │     0 │
│ 2 │ val_accuracy   │ MulticlassAccuracy │      0 │ train │     0 │
│ 3 │ test_accuracy  │ MulticlassAccuracy │      0 │ train │     0 │
└───┴────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 11.5 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 11.5 M                                                                                               
Total estimated model params size (MB): 46                                                                         
Modules in train mode: 75                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
trainer.test(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6071428656578064     │
│         test_loss         │    1.7723591327667236     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.7723591327667236, 'test_acc': 0.6071428656578064}]

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/BongoDev/Project-1/OCR/model_weights.pth")

In [ ]:
image_file = data_module.val_dataset.image_files[0]
pred_text = model.predict_image(image_file)
print(f"Predicted text for {os.path.basename(image_file)}: {pred_text}")

Predicted text for 0b3b.png: 0b3b
